In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [4]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries

from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import data  # to test results when using differentiation


In [3]:
LinearRegression()

LinearRegression()

In [5]:
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import series
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog_predict

series_dict = joblib.load(r'C:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\tests\fixture_sample_multi_series.joblib')
exog_dict = joblib.load(r'C:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\tests\fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

series_2 = pd.DataFrame({'1': pd.Series(np.arange(start=0, stop=50, dtype=float)), 
                         '2': pd.Series(np.arange(start=50, stop=100, dtype=float))})

In [13]:
transformer_exog = ColumnTransformer(
                        [('scale', StandardScaler(), ['exog_1']),
                            ('onehot', OneHotEncoder(), ['exog_2'])],
                        remainder = 'passthrough',
                        verbose_feature_names_out = False
                    )
forecaster = ForecasterAutoregMultiSeries(
                    regressor          = LinearRegression(),
                    lags               = 5,
                    transformer_series = StandardScaler(),
                    transformer_exog   = transformer_exog,
                )
forecaster

============================ 
ForecasterAutoregMultiSeries 
============================ 
Regressor: LinearRegression() 
Lags: [1 2 3 4 5] 
Window features: None 
Window size: 5 
Series names (levels): None 
Series encoding: ordinal 
Series weights: None 
Exogenous included: False 
Exogenous names: None 
Transformer for series: StandardScaler() 
Transformer for exog: ColumnTransformer(remainder='passthrough',
                  transformers=[('scale', StandardScaler(), ['exog_1']),
                                ('onehot', OneHotEncoder(), ['exog_2'])],
                  verbose_feature_names_out=False) 
Weight function included: False 
Differentiation order: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: 
    {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False} 
fit_kwargs: {} 
Creation date: 2024-10-13 12:03:19 
Last fit date: None 
Skforecast version: 0.14.0 
Python version: 3.12.4 
Forecaster id: None

In [ ]:

forecaster.fit(series=series, exog=exog)
predictions = forecaster.predict_interval(steps=5, levels=['1', '2'], exog=exog_predict)
predictions.to_numpy()

In [24]:
series = {
    'l1': pd.Series(np.arange(10, dtype=float)), 
    'l2': pd.Series(np.arange(15, 20, dtype=float)),
    'l3': pd.Series(np.arange(20, 25, dtype=float))
}
series['l1'].loc[3] = np.nan
series['l2'].loc[2] = np.nan
series['l1'].index = pd.date_range("1990-01-01", periods=10, freq='D')
series['l2'].index = pd.date_range("1990-01-05", periods=5, freq='D')
series['l3'].index = pd.date_range("1990-01-03", periods=5, freq='D')

exog = {
    'l1': pd.Series(np.arange(100, 110), name='exog_1', dtype=float),
    'l2': None,
    'l3': pd.DataFrame({'exog_1': np.arange(203, 209, dtype=float),
                        'exog_2': ['a', 'b', 'a', 'b', 'a', 'b']})
}
exog['l1'].index = pd.date_range("1990-01-01", periods=10, freq='D')
exog['l3'].index = pd.date_range("1990-01-03", periods=6, freq='D')

rolling = RollingFeatures(stats=['mean', 'median'], window_sizes=[3, 3])
rolling_2 = RollingFeatures(stats='sum', window_sizes=4)

forecaster = ForecasterAutoregMultiSeries(
    LinearRegression(), lags=3,
    encoding='onehot',
    window_features=[rolling, rolling_2],
    transformer_series=None,
    dropna_from_series=False
)
results = forecaster._create_train_X_y(series=series, exog=exog)
results

c:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:431: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(
c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:553: DataTypeWarning: `exog` may contain only `int`, `float` or `category` dtypes. Most machine learning models do not allow other types of values. Fitting the forecaster may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=ValueTypesExogWarning)
  warnings.warn(
c:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:1014: MissingValuesWarning: NaNs detected in `X_train`. Some regressors do not allow NaN values during training. If you want to drop them, set `forecaster.dropna_from_series = True`. 
 You can suppress 

(            lag_1  lag_2  lag_3  roll_mean_3  roll_median_3  roll_sum_4  l1  \
 1990-01-05    NaN    2.0    1.0          NaN            NaN         NaN   1   
 1990-01-06    4.0    NaN    2.0          NaN            NaN         NaN   1   
 1990-01-07    5.0    4.0    NaN          NaN            NaN         NaN   1   
 1990-01-08    6.0    5.0    4.0          5.0            5.0         NaN   1   
 1990-01-09    7.0    6.0    5.0          6.0            6.0        22.0   1   
 1990-01-10    8.0    7.0    6.0          7.0            7.0        26.0   1   
 1990-01-09   18.0    NaN   16.0          NaN            NaN         NaN   0   
 1990-01-07   23.0   22.0   21.0         22.0           22.0        86.0   0   
 
             l2  l3  exog_1 exog_2  
 1990-01-05   0   0   104.0    NaN  
 1990-01-06   0   0   105.0    NaN  
 1990-01-07   0   0   106.0    NaN  
 1990-01-08   0   0   107.0    NaN  
 1990-01-09   0   0   108.0    NaN  
 1990-01-10   0   0   109.0    NaN  
 1990-01-09   1   0

In [25]:
results[0]

,lag_1,lag_2,lag_3,roll_mean_3,roll_median_3,roll_sum_4,l1,l2,l3,exog_1,exog_2
1990-01-05,NaN,2.0,1.0,NaN,NaN,NaN,1,0,0,104.0,NaN
1990-01-06,4.0,NaN,2.0,NaN,NaN,NaN,1,0,0,105.0,NaN
1990-01-07,5.0,4.0,NaN,NaN,NaN,NaN,1,0,0,106.0,NaN
1990-01-08,6.0,5.0,4.0,5.0,5.0,NaN,1,0,0,107.0,NaN
1990-01-09,7.0,6.0,5.0,6.0,6.0,22.0,1,0,0,108.0,NaN
1990-01-10,8.0,7.0,6.0,7.0,7.0,26.0,1,0,0,109.0,NaN
1990-01-09,18.0,NaN,16.0,NaN,NaN,NaN,0,1,0,NaN,NaN
1990-01-07,23.0,22.0,21.0,22.0,22.0,86.0,0,0,1,207.0,a


In [27]:
results[1].to_numpy()

array([ 4.,  5.,  6.,  7.,  8.,  9., 19., 24.])

In [4]:
end_train = '2003-03-01 23:59:00'

# Simulated exogenous variable
rng = np.random.default_rng(9876)
exog = pd.Series(
    rng.normal(loc=0, scale=1, size=len(data)), index=data.index, name='exog'
)
rolling = RollingFeatures(stats=['mean', 'median'], window_sizes=[5, 5])
rolling_2 = RollingFeatures(stats='sum', window_sizes=[6])

forecaster = ForecasterAutoregDirect(
                    regressor        = LinearRegression(),
                    lags             = [1, 5],
                    window_features  = [rolling, rolling_2],
                    steps            = 6,
                    transformer_y    = None,
                    transformer_exog = None,
                    differentiation  = None
                )
forecaster.fit(y=data.loc[:end_train], exog=exog.loc[:end_train])
X_predict = forecaster.create_predict_X(exog=exog.loc[end_train:])
X_predict

,lag_1,lag_5,roll_mean_5,roll_median_5,roll_sum_6,exog
2003-04-01,2.043333,2.366796,2.261869,2.366796,13.686322,1.161729
2003-05-01,2.043333,2.366796,2.261869,2.366796,13.686322,0.294688
2003-06-01,2.043333,2.366796,2.261869,2.366796,13.686322,-0.439976
2003-07-01,2.043333,2.366796,2.261869,2.366796,13.686322,1.250084
2003-08-01,2.043333,2.366796,2.261869,2.366796,13.686322,1.374969
2003-09-01,2.043333,2.366796,2.261869,2.366796,13.686322,-0.416732


In [8]:
for i, step in enumerate(range(1, forecaster.steps + 1)):
    print(forecaster.regressors_[step].predict(X_predict.iloc[[i]]))

[2.19238587]
[2.23555505]
[2.25654547]
[2.34763646]
[2.31040474]
[2.43900132]


In [10]:
forecaster.predict(steps=[2], exog=exog.loc[end_train:]).to_numpy()

array([2.23555505])

In [11]:
import pytest
import re
import numpy as np
import pandas as pd
from skforecast.model_selection._split import BaseFold

In [12]:
valid_params = {
    "steps": 5,
    "initial_train_size": 100,
    "window_size": 10,
    "differentiation": None,
    "refit": True,
    "fixed_train_size": True,
    "gap": 0,
    "skip_folds": None,
    "allow_incomplete_fold": True,
    "return_all_indexes": False,
    "verbose": True,
}

In [13]:
cv = BaseFold()
params = dict(valid_params)
params["initial_train_size"] = "invalid"
msg = (
    f"`initial_train_size` must be an integer greater than 0 or None. "
    f"Got {params["initial_train_size"]}."
)
with pytest.raises(ValueError, match=msg):
    cv._validate_params(cv_name="TimeSeriesFold", **params)

In [15]:
params is valid_params

False

In [14]:
params

{'steps': 5,
 'initial_train_size': 'invalid',
 'window_size': 10,
 'differentiation': None,
 'refit': True,
 'fixed_train_size': True,
 'gap': 0,
 'skip_folds': None,
 'allow_incomplete_fold': True,
 'return_all_indexes': False,
 'verbose': True}

In [5]:
series = pd.DataFrame({'l1': pd.Series(np.arange(10, dtype=float)), 
                        'l2': pd.Series(np.arange(10, 20, dtype=float))})
series.index = pd.date_range("1990-01-01", periods=10, freq='D')
exog = pd.Series(np.arange(100, 110), name='exog', dtype=float)
exog.index = pd.date_range("1990-01-01", periods=10, freq='D')

rolling = RollingFeatures(stats=['mean', 'median'], window_sizes=[3, 5])
rolling_2 = RollingFeatures(stats='sum', window_sizes=[4])
forecaster = ForecasterAutoregMultiSeries(
    LinearRegression(), lags=None, window_features=[rolling, rolling_2]
)

results = forecaster._create_train_X_y(series=series, exog=exog)
results

c:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:431: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


(            roll_mean_3  roll_median_5  roll_sum_4  _level_skforecast   exog
 1990-01-06          3.0            2.0        10.0                  0  105.0
 1990-01-07          4.0            3.0        14.0                  0  106.0
 1990-01-08          5.0            4.0        18.0                  0  107.0
 1990-01-09          6.0            5.0        22.0                  0  108.0
 1990-01-10          7.0            6.0        26.0                  0  109.0
 1990-01-06         13.0           12.0        50.0                  1  105.0
 1990-01-07         14.0           13.0        54.0                  1  106.0
 1990-01-08         15.0           14.0        58.0                  1  107.0
 1990-01-09         16.0           15.0        62.0                  1  108.0
 1990-01-10         17.0           16.0        66.0                  1  109.0,
 1990-01-06     5.0
 1990-01-07     6.0
 1990-01-08     7.0
 1990-01-09     8.0
 1990-01-10     9.0
 1990-01-06    15.0
 1990-01-07    16.0
 

In [7]:
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import series
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog_predict

rolling = RollingFeatures(stats=['mean', 'median'], window_sizes=4)
transformer_exog = ColumnTransformer(
                        [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                        remainder = 'passthrough',
                        verbose_feature_names_out = False
                    )
forecaster = ForecasterAutoregMultiSeries(
                    regressor          = LGBMRegressor(verbose=-1),
                    lags               = 5,
                    window_features    = rolling,
                    transformer_series = StandardScaler(),
                    transformer_exog   = transformer_exog,
                )
forecaster.fit(series=series, exog=exog)

last_window, exog_values_dict, levels, _, _ = (
    forecaster._create_predict_inputs(steps=5, exog=exog_predict)
)
predictions = forecaster._recursive_predict(
                    steps            = 5,
                    levels           = levels,
                    last_window      = last_window,
                    exog_values_dict = exog_values_dict
                )
predictions

array([[ 0.52368935,  0.88399212],
       [-0.13413796, -0.81145254],
       [ 0.26257161,  0.40407821],
       [ 0.93852676,  0.93243637],
       [ 0.06002111,  0.16197356]])

In [10]:
rolling = RollingFeatures(stats=['mean', 'median'], window_sizes=4)
transformer_exog = ColumnTransformer(
                        [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                        remainder = 'passthrough',
                        verbose_feature_names_out = False
                    )

forecaster = ForecasterAutoregMultiSeries(
                    regressor          = LGBMRegressor(verbose=-1),
                    lags               = 5,
                    window_features    = rolling,
                    transformer_series = StandardScaler(),
                    transformer_exog   = transformer_exog
                )
forecaster.fit(series=series, exog=exog)
predictions = forecaster.predict_bootstrapping(
                    steps                   = 6, 
                    n_boot                  = 10, 
                    exog                    = exog_predict, 
                    use_in_sample_residuals = True
                )
predictions['1'].to_numpy()

array([[0.70414275, 0.33846487, 0.51525768, 0.78238922, 0.53358641,
        0.55496755, 0.49698665, 0.3156552 , 0.70164044, 0.51987002],
       [0.5093462 , 0.10945852, 0.50986314, 0.53323641, 0.50986314,
        0.31517078, 0.54867791, 0.09113636, 0.46379211, 0.41342218],
       [0.86504059, 0.56932758, 0.45218404, 0.44580085, 0.83119047,
        0.43019077, 0.68569118, 0.51340363, 0.61452721, 0.84957527],
       [0.65524631, 0.84194486, 0.93082861, 0.7155279 , 0.83658981,
        1.02306043, 0.6582736 , 0.86211073, 0.56189833, 0.70564327],
       [0.54647555, 0.75265845, 0.43334796, 0.36702491, 0.56762363,
        0.43184972, 0.66142918, 0.47738529, 0.57800408, 0.17483813],
       [0.53343541, 0.56488173, 0.78929977, 0.44786181, 0.49626976,
        0.6850046 , 0.36524591, 0.70909355, 0.34761764, 0.39494035]])

In [14]:
predictions['2'].to_numpy()

array([[0.4703539 , 0.61121897, 0.7456648 , 1.05767197, 0.85702976,
        0.84549833, 0.90211559, 1.05767197, 0.84549833, 0.43576797],
       [0.17638536, 0.36323208, 0.28360047, 0.39694573, 0.42021445,
        0.62085666, 0.21910573, 0.1686658 , 0.03353859, 0.17638536],
       [0.63706768, 0.95491448, 0.84158046, 0.88270255, 0.63313527,
        0.56008641, 0.4801817 , 0.64017749, 0.48757368, 0.70482518],
       [0.88520648, 0.81333217, 0.27723066, 0.44112079, 0.45918293,
        0.57023604, 0.96036122, 0.63876782, 0.81917714, 0.82440159],
       [0.45118834, 0.55605023, 0.84244033, 0.9414499 , 0.55014026,
        0.50114166, 0.25916732, 0.52348578, 0.64211623, 0.49025447],
       [0.49198322, 0.10785838, 0.12577904, 0.12725819, 0.79780591,
        0.54327887, 0.78210044, 0.78482102, 0.19724282, 0.7278225 ]])

In [16]:
pd.Series(np.arange(10, 16, dtype=float)).ravel()

C:\Users\jaesc2\AppData\Local\Temp\ipykernel_12892\685590664.py:1: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  pd.Series(np.arange(10, 16, dtype=float)).ravel()


array([10., 11., 12., 13., 14., 15.])